In [1]:
import cv2
import pandas as pd
import numpy as np
from pathlib import Path
import os
from multiprocessing import pool, cpu_count
from pprint import pprint
from matplotlib import pyplot as plt



In [2]:
def gps2xy(lon, lat):
    # GPS coordinate: 121.0047308 30.8856365最西 121.9724096 30.8997416最东 121.2702538 30.7056483最南 121.3004672 31.5001065最北
    # raster_loc = 1766 685最南 32 742最北 1340 1995最东 1365 192最西
    
    # calculate coefficient k and b
    """
    gpsX = np.array([[103.9925022, 104.088888], [688, 1729]])
    gpsY = np.array([[30.7097699, 30.590786], [856, 2144]])

    diffX = gpsX[:, 1] - gpsX[:, 0]
    kx = diffX[1] / diffX[0]
    bx = gpsX[1, 0] - gpsX[0, 0] * kx

    diffY = gpsY[:, 1] - gpsY[:, 0]
    ky = diffY[1] / diffY[0]
    by = gpsY[1, 0] - gpsY[0, 0] * ky
    print(kx, bx) 
    print(ky, by)
    """
    kx = 1896.250418888416 
    bx = -229280.27146698005
    ky = -2215.827560963539
    by = 69829.24459460145
 
 
    coordX = np.round(kx * lon + bx).astype(int)
    coordY = np.round(ky * lat + by).astype(int)
    # print(coordX, coordY)
    return (coordX, coordY) 
 

In [3]:
def get_region_label(lon, lat):
    map_table = np.loadtxt('region_labeled.csv')
    (coordX, coordY) = gps2xy(lon, lat)
    #print(coordX, coordY)
    labels = np.zeros(lon.shape[0])
    for k in range(len(coordX)):
        try:
            label = map_table[coordX[k], coordY[k]]
            labels[k] = label
        except IndexError:
            labels[k] = 0
    return labels.astype(int)

In [4]:
map_table = np.loadtxt('region_labeled.csv')
print(map_table[-1240,-897] )

60.0


In [5]:
def get_region_code(df):
    l = df.shape[0]
    rc = []
    
    for row in df.itertuples():
        rtype = row.type.split(';')[0]
        if rtype in region_code:
            rc.append(region_code[rtype])
        else:
            rc.append(-1)
    return rc


region_code = {  '教育学校':    0,
            '购物':             1,
            '基础设施':         2,
            '机构团体':         3,
            '公司企业':         4,
            '建筑房产':         5,
            '酒店宾馆':         6,
            '旅游服务':         7,
            '美食':             8,
            '汽车':             9,
            '生活服务':         10,
            '文化场馆':         11,
            '休闲娱乐':         12,
            '银行金融':         13,
            '运动健身':         14,
            '医疗保健服务':     15   }    


In [6]:
def plot_poi():
    poi_src = Path(r'E:\Program Files\jupyter_code\data\poi')
    srcs = poi_src.glob('*')
    for pth in srcs:
        df=pd.read_csv(pth,header=0,encoding = "gb2312")
        c=df
        # c.drop(c.columns[[0,1]], axis=1, inplace=True) # 删除列
        #print(c)
        lat = c.loc[:,'lat']
        lon = c.loc[:,'lng']
        
        region_labels = get_region_label(lon, lat)
        c.loc[:,'region'] = region_labels
        c = c[(c['region'] != 0) & (c['region'] != 1)]
        print(c)
        c.loc[:,'ntype'] = get_region_code(c)
        print(c)
        c.to_csv('poi_total.csv', index=False)

plot_poi()

              lat         lng         type  region
2       31.057080  121.711432    休闲娱乐;旅游景点     150
5       31.275961  121.452373        购物;超市     125
6       31.022801  121.658286    休闲娱乐;户外活动     149
7       31.262445  121.436216    交通运输;港口码头     125
11      31.302132  121.513386      购物;其它购物     142
12      31.218211  121.454123    公司企业;公司企业     116
13      31.326020  121.508899     休闲娱乐;游乐场     126
14      31.264680  121.477181      购物;其它购物     135
15      31.143592  121.653365     休闲娱乐;游乐场     151
17      31.189022  121.437986     休闲娱乐;游乐场     116
18      31.204831  121.608369      购物;母婴儿童     144
19      31.077334  121.493535      购物;母婴儿童     131
20      31.045096  121.467026      购物;服饰鞋包     131
21      31.295337  121.525297    休闲娱乐;户外活动     142
23      31.155741  121.433219      教育学校;培训     116
24      31.165305  121.337022      购物;综合商场      98
25      31.165824  121.337102    建筑房产;商务楼宇      98
26      31.165005  121.337012      购物;其它购物      98
27      31.164665  121.336703  

E:\Program Files\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
E:\Program Files\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


              lat         lng         type  region  ntype
2       31.057080  121.711432    休闲娱乐;旅游景点     150     12
5       31.275961  121.452373        购物;超市     125      1
6       31.022801  121.658286    休闲娱乐;户外活动     149     12
7       31.262445  121.436216    交通运输;港口码头     125     -1
11      31.302132  121.513386      购物;其它购物     142      1
12      31.218211  121.454123    公司企业;公司企业     116      4
13      31.326020  121.508899     休闲娱乐;游乐场     126     12
14      31.264680  121.477181      购物;其它购物     135      1
15      31.143592  121.653365     休闲娱乐;游乐场     151     12
17      31.189022  121.437986     休闲娱乐;游乐场     116     12
18      31.204831  121.608369      购物;母婴儿童     144      1
19      31.077334  121.493535      购物;母婴儿童     131      1
20      31.045096  121.467026      购物;服饰鞋包     131      1
21      31.295337  121.525297    休闲娱乐;户外活动     142     12
23      31.155741  121.433219      教育学校;培训     116      0
24      31.165305  121.337022      购物;综合商场      98      1
25      31.165